In [87]:
from util import local_settings
from openai import OpenAI

print("First LLM API example")
print(f"✅ OpenAI Key loaded ({local_settings.OPENAI_API_KEY[0:-15]}...)")

client = OpenAI(api_key=local_settings.OPENAI_API_KEY)

sk-Ez70Zdl8M6eOe50Uu68DT3BlbkFJgDj6B
First LLM API example
✅ OpenAI Key loaded (sk-Ez70Zdl8M6eOe50Uu68DT3BlbkFJgDj6B...)


In [10]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0, messages=None):
    if not messages:
        messages = [{"role": "user", "content": prompt}]
    else:
        messages.append({"role": "user", "content": prompt})

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return completion.choices[0].message.content

In [3]:
messages = [
    {
        'role': 'system',
        'content': """
            You are ReviewBot, an automated service to generate reviews for places to visit in Lisbon. 
            By pretending you are a person that has visited the place, \
                generate a review based on the provided information about the place. 
            The length of the review must be between 10 and 200 words.
            The rating you give must exist in the dataset. For example, \
                if TerribleRating = 0, then the rating can not be between 0 and 1.

            The dataset includes information about places, their features, a rating and the number of ratings per star.

            Example dataset row:
             - Place: Belém Tower
             - Features: Historical landmark
             - Rating: 4.5/5
             - ExcellentRating: 60 (number of ratings between 4 and 5 stars)
             - VeryGoodRating: 74 (number of ratings between 3 and 4 stars)
             - AverageRating: 32 (number of ratings between 2 and 3 stars)
             - PoorRating: 5 (number of ratings between 1 and 2 stars)
             - TerribleRating: 3 (number of ratings between 0 and 1 stars)
        """
    }
]

In [13]:
prompt = "Belém Tower"

response = get_completion(prompt, messages=messages)
print(response)

messages.append({'role':'assistant', 'content':f"{response}"})

I recently visited Belém Tower in Lisbon and it was truly a historical landmark worth exploring. The architecture is stunning and the views from the top are breathtaking. The intricate details and the rich history behind this tower make it a must-visit attraction. I was impressed by the well-preserved structure and the informative exhibits inside. The only downside was the crowd, but it didn't take away from the overall experience. I highly recommend visiting Belém Tower if you're in Lisbon.


In [4]:
import pandas as pd

# Load the CSV files into DataFrames
trip_df = pd.read_csv('./data/cleanTripLisbon.csv', index_col='Unnamed: 0')
customer_df = pd.read_csv('./data/customer_data.csv')

In [66]:
customer_df.sample(n=1).index[0]

727

In [65]:
customer_df.sample(n=1)

,Username,Email,Password,Age,Planned Trips,Disabilities
951,sharonmann,odavis@example.org,%eW(XgTzA0,20,5,Mobility


In [67]:
reviews_df = pd.DataFrame(columns=['Username', 'Place', 'OverallRating', 'Review', 'Rating'])


,Username,Place,OverallRating,Review,Rating


In [85]:
copy_customer = customer_df.copy().drop(['Email', 'Password', 'Age', 'Disabilities'], axis = 1)
copy_customer['ReviewsNo'] = 0

In [ ]:
def generate_reviews(row, copy_customer):
    if row['ReviewsNo'] < 3:
        for i in range(row['ReviewsNo']-1):
            customer = customer_df.sample(n=1)
            review_dict = {
                'Username': customer['Username'],
                'Place': row['Place'],
                'OverallRating': row['Rating'],
                'Review': #chatgpt,
                'Rating': #chatgpt
            }
            reviews_df.append(review_dict, ignore_index=True)
            # Count if the number of reviews does not exceed the number of trips for the customer
            copy_customer['ReviewsNo'].iloc[customer.index[0]] += 1

In [ ]:
trip_df[trip_df['ReviewsNo'] != 0].apply()

In [69]:
trip_df[trip_df['ReviewsNo'] == 0]

,Link-href,Name,Types,About,Duration,Address,Rating,ReviewsNo,ExcellentRating,VeryGoodRating,AverageRating,PoorRating,TerribleRating,StartTime,EndTime,LunchStart,LunchEnd,DurationMin,DurationMax
90,https://www.tripadvisor.com/Attraction_Review-...,Largo do Carmo,[],NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
630,https://www.tripadvisor.com/Attraction_Review-...,Retrox Vintage,['Antique Shops'],We are a vintage shop specialized in small pro...,Duration: < 1 hour,"Rua dos Anjos 4C Intendente, Lisbon 1150-036 P...",NaN,0,0,0,0,0,0,01:00:00,07:00:00,NaN,NaN,NaN,0 days 01:00:00
1038,https://www.tripadvisor.com/Attraction_Review-...,NaN,[],NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1348,https://www.tripadvisor.com/Attraction_Review-...,Rent4Trail,['Bike Tours'],NaN,NaN,Email,NaN,0,12,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1874,https://www.tripadvisor.com/Attraction_Review-...,Mona - Ideas Store,"['Speciality & Gift Shops', 'Art Galleries']",Mona (it means head in portuguese) is a design...,Duration: < 1 hour,"Rua das Janelas Verdes 70, Lisbon 1200-691 Por...",NaN,0,8,2,0,0,0,NaN,NaN,NaN,NaN,NaN,0 days 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4067,https://www.tripadvisor.com/Attraction_Review-...,Taxi Boat Lisboa,[],Taxi rides and excursions around Lisbon and Ta...,Duration: 1-2 hours,NaN,NaN,0,0,0,0,0,0,00:00:00,23:59:00,NaN,NaN,0 days 01:00:00,0 days 02:00:00
4068,https://www.tripadvisor.com/Attraction_Review-...,Sailboat Aqui Há Peixe,[],NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4069,https://www.tripadvisor.com/Attraction_Review-...,Acervo,[],Founded in March 2016 and based in the former ...,Duration: < 1 hour,"Rua do Machadinho 1, Lisbon 1350-003 Portugal",NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0 days 01:00:00
4070,https://www.tripadvisor.com/Attraction_Review-...,Atrium Bar,[],"a dedicated team, a different space. apart fro...",Duration: More than 3 hours,"Travessa das Prioras 2B, Sacavem 2685-100 Port...",NaN,0,0,0,0,0,0,00:00:00,04:00:00,NaN,NaN,0 days 03:00:00,NaN
